Practicing sequence-2-sequence with transformers in pytorch.

#### Load data and restructure into three pieces
    1. Last k context history pieces
    2. 

In [ ]:
%%shell
mkdir CAsT
cd CAsT
wget https://github.com/daltonj/treccastweb/blob/master/2019/data/evaluation/evaluation_topics_v1.0.json
wget https://github.com/daltonj/treccastweb/blob/master/2019/data/training/train_topics_v1.0.json

--2020-12-10 05:11:45--  https://github.com/daltonj/treccastweb/blob/master/2019/data/evaluation/evaluation_topics_v1.0.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘evaluation_topics_v1.0.json’

evaluation_topics_v     [ <=>                ] 725.45K  --.-KB/s    in 0.07s   

2020-12-10 05:11:45 (10.2 MB/s) - ‘evaluation_topics_v1.0.json’ saved [742864]

--2020-12-10 05:11:45--  https://github.com/daltonj/treccastweb/blob/master/2019/data/training/train_topics_v1.0.json
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘train_topics_v1.0.json’

train_topics_v1.0.j     [  <=>               ] 447.97K  1.37MB/s    in 0.3s    

2020-12-10 05:11:46 (1.37 MB/s) - ‘tra

In [ ]:
# %%shell
# pip install 'urllib3==1.25.4'
# pip install allennlp allennlp-models
# git clone https://github.com/daltonj/treccastweb.git
# wget https://trec.nist.gov/data/cast/2019qrels.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#shared models from Pierre google drive
! cp '/content/drive/MyDrive/STAT_946_Project/GPT2_3_epoch_model_state.bin' .
! cp -r '/content/drive/MyDrive/STAT_946_Project/GPT2_3_epoch_tokenizer_state.bin' .
! git clone https://github.com/daltonj/treccastweb.git
! wget https://trec.nist.gov/data/cast/2019qrels.txt

Mounted at /content/drive
Cloning into 'treccastweb'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 430 (delta 0), reused 0 (delta 0), pack-reused 425
Receiving objects: 100% (430/430), 13.04 MiB | 11.34 MiB/s, done.
Resolving deltas: 100% (186/186), done.
--2020-12-10 05:22:04--  https://trec.nist.gov/data/cast/2019qrels.txt
Resolving trec.nist.gov (trec.nist.gov)... 132.163.4.36, 2610:20:6b01:4::36
Connecting to trec.nist.gov (trec.nist.gov)|132.163.4.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1138032 (1.1M) [text/plain]
Saving to: ‘2019qrels.txt’

2019qrels.txt       100%[===================>]   1.08M  1.59MB/s    in 0.7s    

2020-12-10 05:22:06 (1.59 MB/s) - ‘2019qrels.txt’ saved [1138032/1138032]



In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 13.8MB/s 
     |████████████████████████████████| 890kB 64.0MB/s 
     |████████████████████████████████| 2.9MB 59.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=69076a9b214ccb83207caa8b59f71cdaf629b98a9b7c7b831abac8d0348436c9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
def next_token(input, model, device):
  # To try only can select tokens from input
  model.eval()
  token = model.forward(input_ids=input.to(device))['logits'][-1]
  id = np.argsort(torch.nn.functional.softmax(token).cpu().detach().numpy())[-1]
  new_input = torch.tensor(input.cpu().detach().numpy().tolist() + [id])
  # actual token, id
  print('predict:',tokenizer.decode(torch.tensor(id)))
  return new_input, tokenizer.decode(torch.tensor(id))

Test CAsT


In [ ]:
import json
import time
import datetime
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel,GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler


assessed_queries = set()
with open("/content/2019qrels.txt") as f:
  data = f.readlines()
for line in data:
  qid, _, _, _ = line.strip().split()
  assessed_queries.add(qid)

print("{} queires have relevance assessments".format(len(assessed_queries)))

with open('/content/treccastweb/2019/data/evaluation/evaluation_topics_v1.0.json') as jsonfile:
    data = json.load(jsonfile)


# tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>',
#                                   pad_token='<|pad|>', additional_special_tokens=['<|sep|>','<|go|>'])
tokenizer = GPT2Tokenizer.from_pretrained("/content/GPT2_3_epoch_tokenizer_state.bin")
model = GPT2LMHeadModel.from_pretrained('gpt2')

model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('/content/GPT2_3_epoch_model_state.bin'))
# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()
model = model.to(device)

# Needed since tokens were added to embedding matrix which is used at the input and output of the model.


# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


which_queries='raw_utterance'
count = 0

for conversation in data:
    whole_conversation='<|startoftext|>'
    Conversation_number=conversation['number']
    
    for utterance in conversation['turn']:
        coref_resolutioned= ""
        u_number=utterance['number']
        query_id=str(Conversation_number)+'_'+str(u_number)
        if query_id not in assessed_queries:
          continue
        count += 1        
        if count == 1: 
          tmp_holder = whole_conversation + utterance[which_queries]
          continue
        tmp_holder = tmp_holder + '<|sep|>'
        whole_conversation = tmp_holder + utterance[which_queries] + '<|go|>'
        # print("whole_conversation --> ", whole_conversation)
        
        tensor = torch.tensor(tokenizer(whole_conversation)['input_ids'])
        print('input:',tokenizer.decode(torch.tensor(tensor)))
        i = 0
        tmp_1 = []
        while True:
          tensor,token = next_token(tensor, model, device)
          tmp_1.append(token.strip())
          if token == '<|endoftext|>' or i >= 25:
            break
          i += 1
        
        tmp_2 = " ".join(tmp_1[:-1]).split('<|endoftext|>')[0]
        tmp_holder = tmp_holder + tmp_2
        print("next input --> ", tmp_holder)
        # if count == 9:  break
    # break
print("Processed {} queries".format(count))

173 queires have relevance assessments


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


input: <|startoftext|> What is throat cancer? <|sep|> Is it treatable? <|go|>
predict: Is
predict:  throat
predict:  cancer
predict:  treat
predict: able
predict: ?
predict: <|endoftext|>
next input -->  <|startoftext|>What is throat cancer?<|sep|>Is throat cancer treat able ?
input: <|startoftext|> What is throat cancer? <|sep|> Is throat cancer treat able? <|sep|> Tell me about lung cancer. <|go|>
predict: Tell
predict:  me
predict:  about
predict:  lung
predict:  cancer
predict: .
predict: <|endoftext|>
next input -->  <|startoftext|>What is throat cancer?<|sep|>Is throat cancer treat able ?<|sep|>Tell me about lung cancer .
input: <|startoftext|> What is throat cancer? <|sep|> Is throat cancer treat able? <|sep|> Tell me about lung cancer. <|sep|> What are its symptoms? <|go|>
predict: What
predict:  are
predict:  lung
predict:  cancer
predict: 's
predict:  symptoms
predict: ?
predict: <|endoftext|>
next input -->  <|startoftext|>What is throat cancer?<|sep|>Is throat cancer treat 

RuntimeError: ignored

In [ ]:
arr[0]